In [1]:
import uproot
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import scipy
from tqdm import tqdm
from operator import itemgetter
import pandas as pd
import collections

In [2]:
root_tt = "test/tt/Events/run_01/tag_1_delphes_events.root"
root_ttt = 'test/ttt/Events/run_02/tag_1_delphes_events.root'
root_tttt = "test/tttt/Events/run_01/tag_1_delphes_events.root"
root_3b3l = "signal/3b3l/Events/run_01/tag_1_delphes_events.root"

f_tt = uproot.open(root_tt)
f_ttt = uproot.open(root_ttt)
f_tttt = uproot.open(root_tttt)
f_3b3l = uproot.open(root_3b3l)

In [6]:
pd.DataFrame({
    'Particles': ['d', 'u', 's', 'c', 'b', 't', 'e-', 've',
                     'µ−', 'νµ', 'τ', 'ντ', 'g', 'γ', 'Z0', 'W+', 'h0'],
    'Particle ID': [1, 2, 3, 4, 5, 6, 11, 12, 13, 14, 15, 16, 21, 22, 23, 24, 25],
    'Particle Name': ['down quark', 'up quark', 'strange quark', 'charm quark', 'bottom quark', 
                      'top quark', 'electron', 'electron neutrino', 'muon', 'muon neutrino',
                      'tau', 'tau neutrino', 'gluon', 'photon', 'Z boson', 'W boson', 'Higgs boson']
})

,Particles,Particle ID,Particle Name
0,d,1,down quark
1,u,2,up quark
2,s,3,strange quark
3,c,4,charm quark
4,b,5,bottom quark
5,t,6,top quark
6,e-,11,electron
7,ve,12,electron neutrino
8,µ−,13,muon
9,νµ,14,muon neutrino


In [8]:
l1 = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9])
print([*itemgetter(0, 3)(l1)][0])

1


In [9]:
p['Particle.PID']

<Array [[1, 21, -1, 6, -6, ... 22, 22, 22, 22]] type='10000 * var * int32'>

In [278]:
n1, n2 = 0, 50
nevent = 2
df1 = pd.DataFrame({
        'PID': p['Particle.PID'][nevent],
        'Mother1': p['Particle.M1'][nevent],
        'Mother2': p['Particle.M2'][nevent],
        'Daughter1': p['Particle.D1'][nevent],
        'Daughter2': p['Particle.D2'][nevent],
        'Mass': p['Particle.Mass'][nevent],
        'Energy': p['Particle.E'][nevent],
        'PT': p['Particle.PT'][nevent],
        'Status': p['Particle.Status'][nevent]
})
df1

,PID,Mother1,Mother2,Daughter1,Daughter2,Mass,Energy,PT,Status
0,21,5,-1,3,4,0.00000,129.152145,0.000000,21
1,2,5,-1,15,15,0.33000,879.043884,591.276123,43
2,21,6,-1,3,4,0.00000,1214.644531,0.000000,21
3,6,0,2,8,8,173.00000,1050.464233,224.999130,22
4,-6,0,2,9,9,173.00000,293.332428,224.999130,22
...,...,...,...,...,...,...,...,...,...
1804,22,1800,-1,-1,-1,0.00000,5.728900,4.104572,1
1805,-321,1801,-1,-1,-1,0.49368,14.011093,9.586328,1
1806,211,1801,-1,-1,-1,0.13957,8.620698,6.244608,1
1807,22,1802,-1,-1,-1,0.00000,12.369878,8.380644,1


In [3]:
def BR_cal(f_t, n_t):
    record = []
    wcount = []
    tree = f_t['Delphes;1/Particle']
    p = tree.arrays(['Particle.PID',
                    'Particle.Status',
                    'Particle.M1',
                    'Particle.M2',
                    'Particle.D1',
                    'Particle.D2',
                    'Particle.Mass',
                    'Particle.E',
                    'Particle.PT'])
    for i in tqdm(range(len(p['Particle.PID']))):
        df1 = pd.DataFrame({
            'PID': p['Particle.PID'][i],
            'Mother1': p['Particle.M1'][i],
            'Mother2': p['Particle.M2'][i],
            'Daughter1': p['Particle.D1'][i],
            'Daughter2': p['Particle.D2'][i],
            'Mass': p['Particle.Mass'][i],
            'Energy': p['Particle.E'][i],
            'PT': p['Particle.PT'][i],
            'Status': p['Particle.Status'][i]
        })
        # Find W boson that will decay
        df_w = df1[np.abs(df1.PID) == 24][lambda df: df['Daughter1']!=df['Daughter2']]

        # eliminate any w bosons + others decay combinations
        if len(df_w) > n_t:
            df_w = pd.DataFrame([df_w.iloc[a] if np.abs(df1.loc[df_w.iloc[a]['Daughter1']].PID) != 24 and np.abs(df1.loc[df_w.iloc[a]['Daughter2']].PID) != 24 else df_w.iloc[a].replace({df_w.iloc[a].PID: None}) for a in range(len(df_w))])
            df_w = df_w.dropna()
            
        wcount.append(len(df_w))

        # 1: hadron 2: semi-lepton (e, u) 3: tau 0: others
        for j in range(len(df_w)):
            df_wd1, df_wd2 = df1.loc[df_w.iloc[j]['Daughter1']], df1.loc[df_w.iloc[j]['Daughter2']]
            record.append(1) if 1 <= np.abs(df_wd1.PID) <= 5 and 1 <= np.abs(df_wd2.PID) <= 5 else record.append(2) if 11 <= np.abs(df_wd1.PID) <= 14 and 11 <= np.abs(df_wd2.PID) <= 14 else record.append(3) if 15 <= np.abs(df_wd1.PID) <= 16 and 15 <= np.abs(df_wd2.PID) <= 16 else record.append(0)
        
    Count = collections.Counter(record)
    wCount = collections.Counter(wcount)
    Sum = Count[0] + Count[1] + Count[2] + Count[3]
    df = pd.DataFrame({
            'hadronic decay': Count[1]/Sum,
            'e, u decay': Count[2]/Sum,
            'tau decay': Count[3]/Sum, 
            'others': Count[0]/Sum}, index=[0])    
    return df, wCount

In [4]:
df, wCount = BR_cal(f_tt, 2)

100%|██████████| 10000/10000 [03:43<00:00, 44.74it/s]


In [5]:
wCount

Counter({2: 10000})

In [6]:
df, wCount = BR_cal(f_ttt, 3)

100%|██████████| 10000/10000 [05:11<00:00, 32.15it/s]


In [13]:
wCount

Counter({3: 10000})

In [6]:
df_wCount = BR_cal(f_tttt, 4)

100%|██████████| 10000/10000 [03:24<00:00, 48.86it/s]


,hadronic decay,"e, u decay",tau decay,others
0,0.6808,0.2127,0.1065,0.0


In [ ]:
BR_cal(f_3b3l, 3)

In [33]:
def BR_cal_tt(f_t, CrossSection=598.2, gcs=[243.7, 81.2+80.99, 27.01], n_t=2):
    record = []
    tree = f_t['Delphes;1/Particle']
    p = tree.arrays(['Particle.PID',
                    'Particle.Status',
                    'Particle.M1',
                    'Particle.M2',
                    'Particle.D1',
                    'Particle.D2',
                    'Particle.Mass',
                    'Particle.E',
                    'Particle.PT'])
    for i in tqdm(range(len(p['Particle.PID']))):
        df1 = pd.DataFrame({
            'PID': p['Particle.PID'][i],
            'Mother1': p['Particle.M1'][i],
            'Mother2': p['Particle.M2'][i],
            'Daughter1': p['Particle.D1'][i],
            'Daughter2': p['Particle.D2'][i],
            'Mass': p['Particle.Mass'][i],
            'Energy': p['Particle.E'][i],
            'PT': p['Particle.PT'][i],
            'Status': p['Particle.Status'][i]
        })
        # Find W boson that will decay
        df_w = df1[np.abs(df1.PID) == 24][lambda df: df['Daughter1']!=df['Daughter2']]

        # eliminate any w bosons + others decay combinations
        if len(df_w) > n_t:
            df_w = pd.DataFrame([df_w.iloc[a] if np.abs(df1.loc[df_w.iloc[a]['Daughter1']].PID) != 24 and np.abs(df1.loc[df_w.iloc[a]['Daughter2']].PID) != 24 else df_w.iloc[a].replace({df_w.iloc[a].PID: None}) for a in range(len(df_w))])
            df_w = df_w.dropna()
        
        com = np.zeros(2)
        # 1: hadron 2: semi-lepton (e, u) 3: tau 0: others
        for j in range(len(df_w)):
            df_wd1, df_wd2 = df1.loc[df_w.iloc[j]['Daughter1']], df1.loc[df_w.iloc[j]['Daughter2']]
            com[j] = 1 if 1 <= np.abs(df_wd1.PID) <= 5 and 1 <= np.abs(df_wd2.PID) <= 5 else 2 if 11 <= np.abs(df_wd1.PID) <= 14 and 11 <= np.abs(df_wd2.PID) <= 14 else 3 if 15 <= np.abs(df_wd1.PID) <= 16 and 15 <= np.abs(df_wd2.PID) <= 16 else 0
        
        Count = collections.Counter(com)
        
        record.append(1) if Count[1] == 2 else record.append(2) if Count[1] == 1 and Count[2] ==1 else record.append(3) if Count[2] == 2 else record.append(-1)
    Count2 = collections.Counter(record)
    Sum = Count2[-1] + Count2[1] + Count2[2] + Count2[3]
    df = pd.DataFrame({
            'qq,qq': [Count2[1]/Sum, CrossSection*Count2[1]/Sum, gcs[0]],
            'qq,lvl': [Count2[2]/Sum, CrossSection*Count2[2]/Sum, gcs[1]],
            'lvl,lvl': [Count2[3]/Sum, CrossSection*Count2[3]/Sum, gcs[2]],
            'includes tau': [Count2[-1]/Sum, CrossSection*Count2[-1]/Sum, -1],}, 
        index=['Branchine Ratio',
           'Calculation Cross Section',
           'Generation Cross Section'])     
    return df

In [34]:
df = BR_cal_tt(f_tt)

100%|██████████| 10000/10000 [03:33<00:00, 46.87it/s]


In [35]:
df

,"qq,qq","qq,lvl","lvl,lvl",includes tau
Branchine Ratio,0.45160,0.28830,0.04970,0.21040
Calculation Cross Section,270.14712,172.46106,29.73054,125.86128
Generation Cross Section,243.70000,162.19000,27.01000,-1.00000


In [45]:
def BR_cal_ttt(f_t, CrossSection=0.2035, gcs=[1.043, 0.01422, 0.0428, 0.04302], n_t=3):
    record = []
    tree = f_t['Delphes;1/Particle']
    p = tree.arrays(['Particle.PID',
                    'Particle.Status',
                    'Particle.M1',
                    'Particle.M2',
                    'Particle.D1',
                    'Particle.D2',
                    'Particle.Mass',
                    'Particle.E',
                    'Particle.PT'])
    for i in tqdm(range(len(p['Particle.PID']))):
        df1 = pd.DataFrame({
            'PID': p['Particle.PID'][i],
            'Mother1': p['Particle.M1'][i],
            'Mother2': p['Particle.M2'][i],
            'Daughter1': p['Particle.D1'][i],
            'Daughter2': p['Particle.D2'][i],
            'Mass': p['Particle.Mass'][i],
            'Energy': p['Particle.E'][i],
            'PT': p['Particle.PT'][i],
            'Status': p['Particle.Status'][i]
        })
        # Find W boson that will decay
        df_w = df1[np.abs(df1.PID) == 24][lambda df: df['Daughter1']!=df['Daughter2']]

        # eliminate any w bosons + others decay combinations
        if len(df_w) > n_t:
            df_w = pd.DataFrame([df_w.iloc[a] if np.abs(df1.loc[df_w.iloc[a]['Daughter1']].PID) != 24 and np.abs(df1.loc[df_w.iloc[a]['Daughter2']].PID) != 24 else df_w.iloc[a].replace({df_w.iloc[a].PID: None}) for a in range(len(df_w))])
            df_w = df_w.dropna()
        
        com = np.zeros(n_t)
        # 1: hadron 2: semi-lepton (e, u) 3: tau 0: others
        for j in range(len(df_w)):
            df_wd1, df_wd2 = df1.loc[df_w.iloc[j]['Daughter1']], df1.loc[df_w.iloc[j]['Daughter2']]
            com[j] = 1 if 1 <= np.abs(df_wd1.PID) <= 5 and 1 <= np.abs(df_wd2.PID) <= 5 else 2 if 11 <= np.abs(df_wd1.PID) <= 14 and 11 <= np.abs(df_wd2.PID) <= 14 else 3 if 15 <= np.abs(df_wd1.PID) <= 16 and 15 <= np.abs(df_wd2.PID) <= 16 else 0
        
        Count = collections.Counter(com)
        
        record.append(1) if Count[1] == 3 else record.append(2) if Count[1] == 2 and Count[2] ==1 else record.append(3) if Count[1] == 1 and Count[2] == 2 else record.append(4) if Count[2] == 3 else record.append(-1)
    Count2 = collections.Counter(record)
    Sum = Count2[-1] + Count2[1] + Count2[2] + Count2[3] + Count2[4]
    df = pd.DataFrame({
            'qq,qq,qq': [Count2[1]/Sum, CrossSection*Count2[1]/Sum, gcs[3]],
            'qq,qq,lvl': [Count2[2]/Sum, CrossSection*Count2[2]/Sum, gcs[2]],
            'qq,lvl,lvl': [Count2[3]/Sum, CrossSection*Count2[3]/Sum, gcs[1]],
            'lvl,lvl,lvl': [Count2[4]/Sum, CrossSection*Count2[4]/Sum, gcs[0]],
            'includes tau': [Count2[-1]/Sum, CrossSection*Count2[-1]/Sum, -1],}, 
        index=['Branchine Ratio',
           'Calculation Cross Section',
           'Generation Cross Section'])     
    return df

In [46]:
BR_cal_ttt(f_ttt)

100%|██████████| 10000/10000 [03:09<00:00, 52.66it/s]


,"qq,qq,qq","qq,qq,lvl","qq,lvl,lvl","lvl,lvl,lvl",includes tau
Branchine Ratio,0.307500,0.305900,0.09150,0.008400,0.286700
Calculation Cross Section,0.062576,0.062251,0.01862,0.001709,0.058343
Generation Cross Section,0.043020,0.042800,0.01422,1.043000,-1.000000


In [231]:
df1.iloc[480]

PID            6.000000
Mother1      236.000000
Mother2       -1.000000
Daughter1    608.000000
Daughter2    609.000000
Mass         173.000000
Energy       618.982239
PT           119.700256
Status        62.000000
Name: 480, dtype: float64

In [233]:
df1.iloc[608:610]

,PID,Mother1,Mother2,Daughter1,Daughter2,Mass,Energy,PT,Status
608,24,480,-1,612,612,84.235336,519.713989,60.705818,22
609,5,480,-1,610,611,4.800000,99.268227,67.207657,23


In [279]:
df_top = df1[np.abs(df1.PID) == 6][lambda df: df['Daughter1']!=df['Daughter2']] # Find top quarks that will decay
# check for top quark radiation

re = []
for i in tqdm(range(len(df_top))):
    df_td1 = df1.loc[df_top.iloc[i]['Daughter1']]
    df_td2 = df1.loc[df_top.iloc[i]['Daughter2']]
    if np.abs(df_td1.PID) == 24:
        df_w = df_td1
    else:
        df_w = df_td2
    for i in range(100):
        if df_w.Daughter1==df_w.Daughter2:
            df_w = df1.loc[df_w.Daughter1]
        else:
            df_wd1 = df1.loc[df_w.Daughter1]
            df_wd2 = df1.loc[df_w.Daughter2]
            break
    re.append(df_wd1)
    re.append(df_wd2)
    

100%|██████████| 3/3 [00:00<00:00, 615.12it/s]


In [280]:
df_top

,PID,Mother1,Mother2,Daughter1,Daughter2,Mass,Energy,PT,Status
26,-6,20,-1,33,34,173.0,1037.844360,823.024292,44
480,6,236,-1,608,609,173.0,618.982239,119.700256,62
481,-6,237,-1,625,626,173.0,900.142517,683.849121,62


In [36]:
df_top.iloc[0].name

26

In [193]:
df_top[df_top.PID == 6]

,PID,Mother1,Mother2,Daughter1,Daughter2,Mass,Energy,PT,Status
480,6,236,-1,608,609,173.0,618.982239,119.700256,62


In [281]:
df_top

,PID,Mother1,Mother2,Daughter1,Daughter2,Mass,Energy,PT,Status
26,-6,20,-1,33,34,173.0,1037.844360,823.024292,44
480,6,236,-1,608,609,173.0,618.982239,119.700256,62
481,-6,237,-1,625,626,173.0,900.142517,683.849121,62


In [ ]:
df_top = df1[np.abs(df1.PID) == 6][lambda df: df['Daughter1']!=df['Daughter2']]

In [282]:
df = np.abs(df1.loc[df_top['Daughter1']].PID) != 6

33     False
608     True
625     True
Name: PID, dtype: bool

In [187]:
type(df_top.iloc[1])

pandas.core.series.Series

In [192]:
df = pd.DataFrame([df_top.iloc[i] if np.abs(df1.loc[df_top.iloc[i]['Daughter1']].PID) != 6 and np.abs(df1.loc[df_top.iloc[i]['Daughter2']].PID) != 6 else df_top.iloc[i].replace({df_top.iloc[i].PID: None}) for i in range(len(df_top))])
df = df.dropna()
df

,PID,Mother1,Mother2,Daughter1,Daughter2,Mass,Energy,PT,Status
480,6.0,236.0,-1.0,608.0,609.0,173.0,618.982239,119.700256,62.0
481,-6.0,237.0,-1.0,625.0,626.0,173.0,900.142517,683.849121,62.0


In [68]:
df_t = df_top.iloc[0]
df_t

PID            -6.000000
Mother1        20.000000
Mother2        -1.000000
Daughter1      33.000000
Daughter2      34.000000
Mass          173.000000
Energy       1037.844360
PT            823.024292
Status         44.000000
Name: 26, dtype: float64

In [71]:
df_t['PID']
df_t

SyntaxError: invalid syntax (<ipython-input-71-a79df37375ef>, line 1)

In [44]:
df_top = df_top.drop(df_top.iloc[i].name) if np.abs(df1.loc[df_top.iloc[i]['Daughter1']].PID) != 6 and np.abs(df1.loc[df_top.iloc[i]['Daughter1']].PID) != 6 else df_top for i in range(len(df_top))
df_top

SyntaxError: invalid syntax (<ipython-input-44-df5936822d90>, line 1)

In [10]:
def WDecay(dfd):
    re = []
    for i in range(100):
        if dfd.Daughter1==dfd.Daughter2:
            dfd = df1.loc[dfd.Daughter1]
        else:
            dfd1 = df1.loc[dfd.Daughter1]
            dfd2 = df1.loc[dfd.Daughter2]
            break
    return dfd1, dfd2

In [13]:
len(p['Particle.PID'])

10000

In [40]:
record = []
test = []
for i in tqdm(range(len(p['Particle.PID']))):
#for i in [742, 1696, 2797, 2929, 3119, 3814, 6572, 7448, 8107]:
    df1 = pd.DataFrame({
        'PID': p['Particle.PID'][i],
        'Mother1': p['Particle.M1'][i],
        'Mother2': p['Particle.M2'][i],
        'Daughter1': p['Particle.D1'][i],
        'Daughter2': p['Particle.D2'][i],
        'Mass': p['Particle.Mass'][i],
        'Energy': p['Particle.E'][i],
        'PT': p['Particle.PT'][i],
        'Status': p['Particle.Status'][i]
    })
    # Find top quarks that will decay
    #df_top = df1[np.abs(df1.PID) == 6][lambda df: df['Daughter1']!=df['Daughter2']]
    
    # eliminate any top quarks + others decay combinations
    #if len(df_top) > 2:
        #df_top = pd.DataFrame([df_top.iloc[a] if np.abs(df1.loc[df_top.iloc[a]['Daughter1']].PID) != 6 and np.abs(df1.loc[df_top.iloc[a]['Daughter2']].PID) != 6 else df_top.iloc[a].replace({df_top.iloc[a].PID: None}) for a in range(len(df_top))])
        #df_top = df_top.dropna()
    
    #for j in range(len(df_top)):
        #df_td1, df_td2 = df1.loc[df_top.iloc[j]['Daughter1']], df1.loc[df_top.iloc[j]['Daughter2']]
        #df_w = df_td1 if np.abs(df_td1.PID) == 24 else df_td2
        #df_wd1, df_wd2 = WDecay(df_w)
        
    df_w = df1[np.abs(df1.PID) == 24][lambda df: df['Daughter1']!=df['Daughter2']]
    
    for j in range(len(df_w)):
        df_wd1, df_wd2 = df1.loc[df_w.iloc[j]['Daughter1']], df1.loc[df_w.iloc[j]['Daughter2']]

        # Fully Hadronic Decay
        if 1 <= np.abs(df_wd1.PID) <= 6 and 1 <= np.abs(df_wd2.PID) <= 6:
            if 1 in {np.abs(df_wd1.PID), np.abs(df_wd2.PID)} and 2 in {np.abs(df_wd1.PID), np.abs(df_wd2.PID)}:
                record.append('ud')
            elif 3 in {np.abs(df_wd1.PID), np.abs(df_wd2.PID)} and 4 in {np.abs(df_wd1.PID), np.abs(df_wd2.PID)}:
                record.append('cs')
            elif 4 in {np.abs(df_wd1.PID), np.abs(df_wd2.PID)} and 5 in {np.abs(df_wd1.PID), np.abs(df_wd2.PID)}:
                record.append('cb')
            elif 1 in {np.abs(df_wd1.PID), np.abs(df_wd2.PID)} and 4 in {np.abs(df_wd1.PID), np.abs(df_wd2.PID)}:
                record.append('cd')
            elif 2 in {np.abs(df_wd1.PID), np.abs(df_wd2.PID)} and 3 in {np.abs(df_wd1.PID), np.abs(df_wd2.PID)}:
                record.append('us')
            else: 
                record.append('had_others')

        # Semi-leptonic Decay
        elif 11 <= np.abs(df_wd1.PID) <= 16 and 11 <= np.abs(df_wd2.PID) <= 16:
            if 11 in {np.abs(df_wd1.PID), np.abs(df_wd2.PID)} and 12 in {np.abs(df_wd1.PID), np.abs(df_wd2.PID)}:
                record.append('eve')
            elif 13 in {np.abs(df_wd1.PID), np.abs(df_wd2.PID)} and 14 in {np.abs(df_wd1.PID), np.abs(df_wd2.PID)}:
                record.append('uvu')
            elif 15 in {np.abs(df_wd1.PID), np.abs(df_wd2.PID)} and 16 in {np.abs(df_wd1.PID), np.abs(df_wd2.PID)}:
                record.append('tavta')
            else:
                record.append('lep_others')
                
        # zw, hw, aw
        elif np.abs(df_wd1.PID) == 24 or np.abs(df_wd2.PID) == 24:
            if np.abs(df_wd1.PID) + np.abs(df_wd2.PID) == 46:
                record.append('aw')
            elif np.abs(df_wd1.PID) + np.abs(df_wd2.PID) == 47:
                record.append('zw')
            elif np.abs(df_wd1.PID) + np.abs(df_wd2.PID) == 49:
                record.append('hw')
            else:
                record.append('w_others')
                
        else:
            record.append('others')

100%|██████████| 10000/10000 [04:09<00:00, 40.01it/s]


In [49]:
Count = collections.Counter(record)
Sum = Count['ud'] + Count['eve'] + Count['cs'] + Count['cd'] + Count['uvu'] + Count['aw'] + Count['us'] + Count['tavta'] + Count['cb']
df = pd.DataFrame({
    'ud~, u~b': Count['ud']/Sum,
    'us~, u~s': Count['us']/Sum,
    'cd~, c~d': Count['cd']/Sum,
    'cs~, c~s': Count['cs']/Sum,
    'cb~, c~b': Count['cb']/Sum,
    'Hadronic': (Count['ud']+Count['cs']+Count['cd']+Count['us']+Count['cb'])/(Sum-Count['aw']),
    'e ve': Count['eve']/Sum,
    'μ vμ': Count['uvu']/Sum,
    'τ vτ': Count['tavta']/Sum,
    'Leptonic': (Count['eve']+Count['uvu']+Count['tavta'])/(Sum-Count['aw']),
    'γW+, γW-': Count['aw']/Sum,
    'Z0W+': Count['zw']/Sum,
    'hW+, hW-': Count['hw']/Sum}, index=[0])    
df

,"ud~, u~b","us~, u~s","cd~, c~d","cs~, c~s","cb~, c~b",Hadronic,e ve,μ vμ,τ vτ,Leptonic,"γW+, γW-",Z0W+,"hW+, hW-"
0,0.295466,0.015595,0.014895,0.297521,0.000514,0.6682,0.102489,0.103282,0.104076,0.3318,0.066162,0.0,0.0


In [30]:
pd.DataFrame(test)

,PID,Mother1,Mother2,Daughter1,Daughter2,Mass,Energy,PT,Status
808,24.0,639.0,-1.0,812.0,812.0,78.944527,332.746521,97.364044,22.0
824,-5.0,812.0,-1.0,828.0,828.0,4.800000,90.707855,10.073299,23.0
825,4.0,812.0,-1.0,826.0,827.0,1.500000,240.937943,104.277840,23.0
1220,-24.0,943.0,-1.0,1224.0,1224.0,83.972778,140.425369,86.670631,22.0
1269,5.0,1224.0,-1.0,1273.0,1273.0,4.800000,85.085945,69.352081,23.0
1270,-4.0,1224.0,-1.0,1271.0,1272.0,1.500000,50.938435,47.926216,23.0
753,24.0,615.0,-1.0,763.0,764.0,80.658806,325.226593,185.384369,22.0
763,-5.0,753.0,-1.0,767.0,767.0,4.800000,119.731262,93.047539,23.0
764,4.0,753.0,-1.0,765.0,766.0,1.500000,205.495331,99.929688,23.0
725,24.0,578.0,-1.0,729.0,729.0,78.923111,296.768036,223.138763,22.0


In [272]:
print(f'problematic event: {test}')
[742, 1696, 2797, 2929, 3119, 3814, 6572, 7448, 8107]

[741, 1696, 2797, 2929, 3119, 3814, 6572, 7448, 8107]

In [255]:
record1 = record

In [259]:
r = collections.Counter(record1)

In [266]:
r[1]+r[2]+r[3]+r[4]+r[5]+r[6]+r[7]+r[8]

20000

In [267]:
r[2]/20000

0.6222

In [28]:
nevents = 20000
pd.DataFrame({
    'hadronic decay': r[]/nevents,
    'semi-leptonic decay': r[1]/nevents,
    'w->aw': r[3]/nevents, 
    'w->zw': r[4]/nevents, 
    'w->hw': r[5]/nevents, 
    'w->tavta': r[6]/nevents, 
    'w->tb': r[7]/nevents, 
    'others': r[8]/nevents}, index=[0])

,hadronic decay,semi-leptonic decay,w->aw,w->zw,w->hw,w->tavta,w->tb,others
0,0.22035,0.6682,0.11145,0.0,0.0,0.0,0.0,0.0


In [26]:
record = []
test = []
for i in tqdm(range(len(p['Particle.PID']))):
    df1 = pd.DataFrame({
        'PID': p['Particle.PID'][i],
        'Mother1': p['Particle.M1'][i],
        'Mother2': p['Particle.M2'][i],
        'Daughter1': p['Particle.D1'][i],
        'Daughter2': p['Particle.D2'][i],
        'Mass': p['Particle.Mass'][i],
        'Energy': p['Particle.E'][i],
        'PT': p['Particle.PT'][i],
        'Status': p['Particle.Status'][i]
    })
    # Find W boson that will decay
    df_w = df1[np.abs(df1.PID) == 24][lambda df: df['Daughter1']!=df['Daughter2']]
    
    # eliminate any w bosons + others decay combinations
    if len(df_w) > 2:
        df_w = pd.DataFrame([df_w.iloc[a] if np.abs(df1.loc[df_w.iloc[a]['Daughter1']].PID) != 24 and np.abs(df1.loc[df_w.iloc[a]['Daughter2']].PID) != 24 else df_w.iloc[a].replace({df_w.iloc[a].PID: None}) for a in range(len(df_w))])
        df_w = df_w.dropna()
        
    # 1: hadron 2: semi-lepton (e, u) 3: tau 0: others
    for j in range(len(df_w)):
        df_wd1, df_wd2 = df1.loc[df_w.iloc[j]['Daughter1']], df1.loc[df_w.iloc[j]['Daughter2']]
        record.append(1) if 1 <= np.abs(df_wd1.PID) <= 5 and 1 <= np.abs(df_wd2.PID) <= 5 else record.append(2) if 11 <= np.abs(df_wd1.PID) <= 14 and 11 <= np.abs(df_wd2.PID) <= 14 else record.append(3) if 15 <= np.abs(df_wd1.PID) <= 16 and 15 <= np.abs(df_wd2.PID) <= 16 else record.append(0)

100%|██████████| 10000/10000 [03:46<00:00, 44.14it/s]


In [24]:
r = collections.Counter(record)
print(r)
Sum = r[0]+r[1]+r[2]+r[3]
pd.DataFrame({
    'hadronic decay': r[1]/Sum,
    'e, u decay': r[2]/Sum,
    'tau decay': r[3]/Sum, 
    'others': r[0]/Sum}, index=[0])

Counter({1: 13364, 2: 4407, 3: 2229})


,hadronic decay,"e, u decay",tau decay,others
0,0.6682,0.22035,0.11145,0.0


In [27]:
r = collections.Counter(record)
print(r)
Sum = r[0]+r[1]+r[2]+r[3]
pd.DataFrame({
    'hadronic decay': r[1]/Sum,
    'e, u decay': r[2]/Sum,
    'tau decay': r[3]/Sum, 
    'others': r[0]/Sum}, index=[0])

Counter({1: 13364, 2: 4407, 3: 2229})


,hadronic decay,"e, u decay",tau decay,others
0,0.6682,0.22035,0.11145,0.0


In [20]:
l1 = []
a, b = 1, 2
l1.append(1) if a < b else 
l1

SyntaxError: invalid syntax (<ipython-input-20-39689b5928da>, line 3)

In [156]:
Decay(df1.loc[621])

(PID           -15.000000
 Mother1       621.000000
 Mother2        -1.000000
 Daughter1    1378.000000
 Daughter2    1380.000000
 Mass            1.776820
 Energy        132.243530
 PT             96.649452
 Status          2.000000
 Name: 708, dtype: float64,
 PID           16.000000
 Mother1      621.000000
 Mother2       -1.000000
 Daughter1     -1.000000
 Daughter2     -1.000000
 Mass           0.000000
 Energy        58.774281
 PT            57.884163
 Status         1.000000
 Name: 709, dtype: float64)

In [87]:
df_w = df1[np.abs(df1.PID) == 24][lambda df: df['Daughter1']!=df['Daughter2']]
df_w

,PID,Mother1,Mother2,Daughter1,Daughter2,Mass,Energy,PT,Status
606,24,602,-1,640,641,81.092484,218.849503,138.406982,52
617,24,613,-1,621,622,79.878952,191.017807,153.943024,52
621,24,617,-1,708,709,79.878952,191.017807,153.943024,51
624,-24,483,-1,710,711,80.208969,405.411591,211.366150,22
630,-24,626,-1,712,713,79.506920,193.292496,127.410492,52


481    602
482    613
483    624
484    626
Name: Daughter1, dtype: int32

In [86]:
df1.iloc[710:720]

,PID,Mother1,Mother2,Daughter1,Daughter2,Mass,Energy,PT,Status
710,13,624,-1,-1,-1,0.10566,397.872894,214.941986,1
711,-14,624,-1,-1,-1,0.00000,7.538692,7.342656,1
712,1,630,-1,716,716,0.33000,155.919174,97.847984,23
713,-2,630,-1,714,715,0.33000,37.373322,37.370731,23
714,-2,713,-1,726,727,0.33000,54.035969,48.022587,51
715,21,713,-1,719,719,0.00000,6.502242,0.865196,51
716,1,712,-1,717,718,0.33000,132.754288,83.310745,52
717,1,716,-1,725,725,0.33000,125.040733,77.517975,51
718,21,716,-1,720,721,0.00000,7.808661,5.868306,51
719,21,715,-1,722,722,0.00000,6.407131,0.852540,52


In [28]:
df_1 = df1.loc[lambda df: df['Daughter1']==-1, :]
df_1 = df_1.loc[lambda df: df['Daughter2']==-1, :]
df_1 = df_1.loc[lambda df: df['PID']!=22, :]
df_1

,PID,Mother1,Mother2,Daughter1,Daughter2,Mass,Energy,PT,Status
502,-16,499,-1,-1,-1,0.00000,38.154037,31.367920,1
660,-211,654,658,-1,-1,0.13957,48.570553,5.426994,1
664,211,496,497,-1,-1,0.13957,10.492217,3.149364,1
665,-211,496,497,-1,-1,0.13957,1.886016,0.832973,1
674,-321,667,672,-1,-1,0.49368,7.282328,4.741437,1
...,...,...,...,...,...,...,...,...,...
1529,-321,1516,-1,-1,-1,0.49368,3.313641,3.269778,1
1530,211,1516,-1,-1,-1,0.13957,2.462548,2.458534,1
1531,211,1517,-1,-1,-1,0.13957,0.430779,0.392579,1
1532,-211,1517,-1,-1,-1,0.13957,1.031316,1.011127,1


In [39]:
len(df_1.loc[lambda df: df['PID']==11, :])
df_1[np.abs(df_1.PID) == 11].shape[0]

4

In [44]:
nelectron = df_1[np.abs(df_1.PID) == 11].shape[0]
nnelectron = df_1[np.abs(df_1.PID) == 12].shape[0]
nmuon = df_1[np.abs(df_1.PID) == 13].shape[0]
nnmuon = df_1[np.abs(df_1.PID) == 14].shape[0]
ntau = df_1[np.abs(df_1.PID) == 15].shape[0]
nntau = df_1[np.abs(df_1.PID) == 16].shape[0]
print(f'electrons: {nelectron}, electron neutrino: {nnelectron}, muon: {nmuon}, nnmuon: {nnmuon} \
 Tau:{ntau}, Tau neutrino: {nntau}')

electrons: 4, electron neutrino: 2, muon: 0, nnmuon: 0  Tau:0, Tau neutrino: 4


In [56]:
df1 = dft.loc[lambda df: df['D1']!=df['D2'], :]
df2 = df1.loc[lambda df: df['PID'] != 21, :]
df2

,PID,Status,M1,M2,D1,D2
2,-4,21,7,-1,3,4
50,4,41,96,-1,39,40
61,1,31,126,-1,64,65
100,2,41,107,-1,94,95
112,-4,44,106,-1,235,236
...,...,...,...,...,...,...
1523,111,2,1508,-1,1536,1537
1524,213,2,1511,-1,1538,1539
1525,111,2,1511,-1,1540,1541
1533,111,2,1517,-1,1542,1543
